<a href="https://colab.research.google.com/github/kvanoost/AGP/blob/main/CB_LU_tiles_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import geemap
import ee

ee.Initialize(project='tropsedslu')

Map = geemap.Map()

# Load your 5-class land cover image
asset_img = ee.Image('projects/tropsedslu/assets/KasaiBainLU/LANDUSE_KASAI_2017-18_SATEMBED')

# Same visualization parameters as in GEE JS
lc_vis = {
    'min': 1,
    'max': 5,
    'palette': [
        '#b30000', # 1 urban - dark red
        '#1f78b4', # 2 water - blue
        '#006400', # 3 forest - dark green
        '#ffff99', # 4 savannah/grassland - yellow
        '#fdbf6f'  # 5 cropland - orange
    ]
}

Map.addLayer(asset_img, lc_vis, 'Land Cover 5 classes')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
import ee
import geemap

ee.Initialize(project='tropsedslu')

# -------------------------------------------------------
# 0. INPUTS
# -------------------------------------------------------
training_fc = ee.FeatureCollection('projects/tropsedslu/assets/CB/Basins/Kasai_refdata')
cb_outline  = ee.FeatureCollection('projects/tropsedslu/assets/CB/Basins/CB_outline').geometry()

class_property = 'land_cover'
embed_col = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')

year_start = 2021
year_end   = 2023
scale      = 10
N_TOP      = 20
N_TREES    = 300

# -------------------------------------------------------
# 1. PREP TRAINING POINTS
# -------------------------------------------------------
# Make labels numeric
def make_numeric_label(f):
    return f.set(class_property, ee.Number.parse(f.get(class_property)))

training_fc = training_fc.map(make_numeric_label)

# Keep only features with a valid label
pts = training_fc.filter(ee.Filter.notNull([class_property]))

print('Training points (server-side):', pts.size().getInfo())

# Use Congo Basin outline as region for embeddings
region = cb_outline

# -------------------------------------------------------
# 2. EMBEDDING PREDICTOR IMAGE (MEAN 2021–2023)
# -------------------------------------------------------
embed_img = (embed_col
    .filter(ee.Filter.calendarRange(year_start, year_end, 'year'))
    .filterBounds(region)
    .mean()
    .clip(region)
)

bands = embed_img.bandNames()
print('Total embedding bands:', bands.size().getInfo())

# -------------------------------------------------------
# 3. SAMPLE TRAINING DATA
# -------------------------------------------------------
samples = embed_img.sampleRegions(
    collection=pts,
    properties=[class_property],
    scale=scale
).filter(ee.Filter.notNull(bands))

samples = samples.randomColumn('split', seed=0)
train_samples = samples.filter('split < 0.7')
valid_samples = samples.filter('split >= 0.7')

print('Train samples:', train_samples.size().getInfo())
print('Valid samples:', valid_samples.size().getInfo())

# -------------------------------------------------------
# 4. FULL MODEL (ALL BANDS) + VARIABLE IMPORTANCE
# -------------------------------------------------------
rf_full = ee.Classifier.smileRandomForest(numberOfTrees=N_TREES).train(
    features=train_samples,
    classProperty=class_property,
    inputProperties=bands
)

# Variable importance
explanation = rf_full.explain()
importance_dict = ee.Dictionary(explanation.get('importance'))

band_names = importance_dict.keys()

def band_to_feature(b):
    b = ee.String(b)
    return ee.Feature(None, {
        'band': b,
        'importance': importance_dict.get(b)
    })

importance_fc = ee.FeatureCollection(band_names.map(band_to_feature))
sorted_fc = importance_fc.sort('importance', False)
top_fc = sorted_fc.limit(N_TOP)

top_bands = ee.List(top_fc.aggregate_array('band'))
print('Top-20 bands:', top_bands.getInfo())

# -------------------------------------------------------
# 5. REDUCED RF MODEL WITH TOP-20 BANDS
# -------------------------------------------------------
rf_top = ee.Classifier.smileRandomForest(numberOfTrees=N_TREES).train(
    features=train_samples,
    classProperty=class_property,
    inputProperties=top_bands
)

# (Optional) quick check of validation accuracy
conf_top = valid_samples.classify(rf_top).errorMatrix(class_property, 'classification')
print('TOP-20 model OA:', conf_top.accuracy().getInfo())
print('TOP-20 model Kappa:', conf_top.kappa().getInfo())

# Predictor image restricted to top bands
predictors_top = embed_img.select(top_bands)

# -------------------------------------------------------
# 6. BUILD 1° GRID OVER CONGO BASIN OUTLINE
# -------------------------------------------------------
bounds = cb_outline.bounds()
coords = ee.List(bounds.coordinates().get(0))

xs = coords.map(lambda c: ee.Number(ee.List(c).get(0)))
ys = coords.map(lambda c: ee.Number(ee.List(c).get(1)))

min_lon = ee.Number(xs.reduce(ee.Reducer.min())).floor()
max_lon = ee.Number(xs.reduce(ee.Reducer.max())).ceil()
min_lat = ee.Number(ys.reduce(ee.Reducer.min())).floor()
max_lat = ee.Number(ys.reduce(ee.Reducer.max())).ceil()

lon_seq = ee.List.sequence(min_lon, max_lon.subtract(1), 1)
lat_seq = ee.List.sequence(min_lat, max_lat.subtract(1), 1)

def make_cell(lon):
    lon = ee.Number(lon)
    def inner(lat):
        lat = ee.Number(lat)
        rect = ee.Geometry.Rectangle(
            [lon, lat, lon.add(1), lat.add(1)], proj=None, geodesic=False
        )
        cell_geom = rect.intersection(cb_outline, 1)
        # area in m² (might be 0 if outside basin)
        area = cell_geom.area(1)
        return ee.Feature(cell_geom, {
            'lon': lon,
            'lat': lat,
            'area_m2': area
        })
    return ee.FeatureCollection(lat_seq.map(inner))

grid_fc_raw = ee.FeatureCollection(lon_seq.map(make_cell)).flatten()
grid_fc = grid_fc_raw.filter(ee.Filter.gt('area_m2', 0))  # keep only cells overlapping basin

print('Number of 1° grid cells over Congo Basin:', grid_fc.size().getInfo())

# -------------------------------------------------------
# 7. CLASSIFY FIRST 3 GRID CELLS WITH THE TOP-20 RF
# -------------------------------------------------------
grid_list = grid_fc.toList(3)  # first 3 cells

classified_tiles = []
for i in range(3):
    cell = ee.Feature(grid_list.get(i))
    geom = cell.geometry()
    preds_tile = predictors_top.clip(geom)
    classified = preds_tile.classify(rf_top)
    classified_tiles.append(classified)

print('Classified 3 tiles.')

# -------------------------------------------------------
# 8. VIEW RESULTS IN GEEMAP
# -------------------------------------------------------
Map = geemap.Map()
Map.centerObject(cb_outline, 5)

# Congo Basin outline
Map.addLayer(cb_outline, {}, 'Congo Basin outline')

# Add the 3 classified tiles, each as a separate layer
palette = ['red', 'blue', 'darkgreen', 'yellow', 'orange']  # [Built, Water, Forest, Grassland, Cropland]

for i, img in enumerate(classified_tiles):
    Map.addLayer(
        img,
        {'min': 1, 'max': 5, 'palette': palette},
        f'Classified tile {i+1}'
    )

# Optional: see the grid cells
Map.addLayer(grid_fc, {'color': 'white'}, '1-degree grid', False)

Map

# -------------------------------------------------------
# 9. EXPORT THE 3 CLASSIFIED TILES AS ASSETS
# -------------------------------------------------------

output_folder = 'projects/tropsedslu/assets/CB/ModelTests/Top20_RF_tiles'

for i, img in enumerate(classified_tiles):

    # Get geometry from grid tiles
    cell = ee.Feature(grid_list.get(i))
    geom = cell.geometry()

    asset_id = f'{output_folder}/tile_{i+1}'

    task = ee.batch.Export.image.toAsset(
        image=img,
        description=f'Export_Top20_RF_tile_{i+1}',
        assetId=asset_id,
        region=geom,
        scale=10,
        maxPixels=1e13
    )
    task.start()
    print(f'Export started for tile {i+1}: {asset_id}')



Training points (server-side): 3734
Total embedding bands: 64
